In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray
import rioxarray as rxr
import glob
from datetime import datetime
import utm
from pyproj import Proj, Transformer
import os

In [2]:
times_GOES = pd.read_csv('../../GOES_tifs/GOES_times.csv')

In [3]:
times_GOES.value = (times_GOES.value/1000).apply(datetime.fromtimestamp)

In [4]:
times_GOES

,system:index,value,.geo
0,0,2022-12-31 22:46:17.300,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,1,2023-01-01 10:46:17.400,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,2,2023-01-01 22:46:17.400,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,3,2023-01-02 10:46:17.400,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,4,2023-01-02 22:46:17.400,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...
720,720,2023-12-28 10:46:17.100,"{""type"":""MultiPoint"",""coordinates"":[]}"
721,721,2023-12-28 22:46:17.100,"{""type"":""MultiPoint"",""coordinates"":[]}"
722,722,2023-12-29 10:46:17.200,"{""type"":""MultiPoint"",""coordinates"":[]}"
723,723,2023-12-29 22:46:17.200,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [5]:
times_Landsat = pd.read_csv('../../Landsat_tifs/Landsat_times.csv')

In [6]:
times_Landsat.value = (times_Landsat.value/1000).apply(datetime.fromtimestamp)

In [7]:
times_Landsat

,system:index,value,.geo
0,0,2023-01-11 10:46:48.979,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,1,2023-02-28 10:46:30.544,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,2,2023-03-16 11:46:23.913,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,3,2023-04-01 11:46:13.064,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,4,2023-04-17 11:45:58.932,"{""type"":""MultiPoint"",""coordinates"":[]}"
5,5,2023-05-03 11:45:53.830,"{""type"":""MultiPoint"",""coordinates"":[]}"
6,6,2023-05-19 11:45:43.340,"{""type"":""MultiPoint"",""coordinates"":[]}"
7,7,2023-06-04 11:45:48.086,"{""type"":""MultiPoint"",""coordinates"":[]}"
8,8,2023-06-20 11:45:55.135,"{""type"":""MultiPoint"",""coordinates"":[]}"
9,9,2023-07-06 11:46:07.055,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [9]:
geotiff_list = glob.glob('../../GOES_tifs/*.tif')
geotiff_list

['../../GOES_tifs/GOES_image_22.tif',
 '../../GOES_tifs/GOES_image_23.tif',
 '../../GOES_tifs/GOES_image_21.tif',
 '../../GOES_tifs/GOES_image_595.tif',
 '../../GOES_tifs/GOES_image_436.tif',
 '../../GOES_tifs/GOES_image_24.tif',
 '../../GOES_tifs/GOES_image_30.tif',
 '../../GOES_tifs/GOES_image_180.tif',
 '../../GOES_tifs/GOES_image_340.tif',
 '../../GOES_tifs/GOES_image_25.tif',
 '../../GOES_tifs/GOES_image_27.tif',
 '../../GOES_tifs/GOES_image_627.tif',
 '../../GOES_tifs/GOES_image_26.tif',
 '../../GOES_tifs/GOES_image_244.tif',
 '../../GOES_tifs/GOES_image_468.tif',
 '../../GOES_tifs/GOES_image_722.tif',
 '../../GOES_tifs/GOES_image_690.tif',
 '../../GOES_tifs/GOES_image_308.tif',
 '../../GOES_tifs/GOES_image_500.tif',
 '../../GOES_tifs/GOES_image_276.tif',
 '../../GOES_tifs/GOES_image_659.tif',
 '../../GOES_tifs/GOES_image_117.tif',
 '../../GOES_tifs/GOES_image_212.tif',
 '../../GOES_tifs/GOES_image_372.tif',
 '../../GOES_tifs/GOES_image_28.tif',
 '../../GOES_tifs/GOES_image_29.ti

In [10]:
def sort_func(s):
    return int(s.split('image_')[1].split('.tif')[0])

In [11]:
geotiff_list = sorted(geotiff_list, key=sort_func)
geotiff_list

['../../GOES_tifs/GOES_image_21.tif',
 '../../GOES_tifs/GOES_image_22.tif',
 '../../GOES_tifs/GOES_image_23.tif',
 '../../GOES_tifs/GOES_image_24.tif',
 '../../GOES_tifs/GOES_image_25.tif',
 '../../GOES_tifs/GOES_image_26.tif',
 '../../GOES_tifs/GOES_image_27.tif',
 '../../GOES_tifs/GOES_image_28.tif',
 '../../GOES_tifs/GOES_image_29.tif',
 '../../GOES_tifs/GOES_image_30.tif',
 '../../GOES_tifs/GOES_image_117.tif',
 '../../GOES_tifs/GOES_image_149.tif',
 '../../GOES_tifs/GOES_image_180.tif',
 '../../GOES_tifs/GOES_image_212.tif',
 '../../GOES_tifs/GOES_image_244.tif',
 '../../GOES_tifs/GOES_image_276.tif',
 '../../GOES_tifs/GOES_image_308.tif',
 '../../GOES_tifs/GOES_image_340.tif',
 '../../GOES_tifs/GOES_image_372.tif',
 '../../GOES_tifs/GOES_image_404.tif',
 '../../GOES_tifs/GOES_image_436.tif',
 '../../GOES_tifs/GOES_image_468.tif',
 '../../GOES_tifs/GOES_image_500.tif',
 '../../GOES_tifs/GOES_image_564.tif',
 '../../GOES_tifs/GOES_image_595.tif',
 '../../GOES_tifs/GOES_image_627.ti

In [12]:
def to_binary_string(num):
    if np.isnan(num):
        return np.NaN
    else:
        return f'{int(num):b}'

In [13]:
to_binary_string(21952)

'101010111000000'

In [14]:
# Define UTM18N projection (WGS84 datum)
global utm_proj 
utm_proj = Proj(proj="utm", zone=18, datum="WGS84", northern=True)

In [65]:
"""
Processing of individual .tif files.

Performs a variety of tasks on the data to make it more easy to read and understand.

Attributes:
    tif (str): Path where tif file is located.
    time (datetime or str): Date and time of when the data was collected.
    name (str): Desired name of output file. Just the name, leave out the ".format" part at the end.
    coord_bounds (tuple or list, optional): Coordinate bounds if you wish to filter the data by location. Order should be
                                    (longitude minimum, longitude maximum, latitude minimum, latitude maximum).
    save_as (str, optional): Format to save the processed data in. For now, only 'nc' is built-in. Anything else
                                    simply returns the data.
"""
def process_Landsat_Sentinel_tif(tif, time, name, coord_bounds=None, save_as='nc'):
    #########################################################################################################
    # Open file and rename variables
    dsLS = rxr.open_rasterio(tif)
    stackedLS = dsLS.stack(utm_coords=('x', 'y'))
    geotiff_dsLS = stackedLS.to_dataset('band')
    
    def stacked_to_latlon(pt):
        return utm_proj(pt[0], pt[1], inverse=True)

    ptsLS = list(map(stacked_to_latlon, geotiff_dsLS.stacked_dim.values))
    geotiff_dsLS.coords['latlon_coords'] = (['latlons', 'coords'], ptsLS)
    
    geotiff_dsLS = geotiff_dsLS.rename({1:'Landsat_Blue_Vis_Sfc_Reflectance', 2:'Landsat_Red_Vis_Sfc_Reflectance',
                                        3:'Landsat_NIR_Sfc_Reflectance', 4:'Landsat_SWIR1_Sfc_Reflectance',
                                        5:'Landsat_SWIR2_Sfc_Reflectance', 6:'Landsat_LST',
                                        7:'Landsat_Cloud_Mask', 8:'VV_SAR', 9:'VH_SAR', 10:'SAR_angle'})

    #########################################################################################################
    # Convert cloud mask integers to binary strings
    mask = xr.DataArray([[to_binary_string(x) for x in line] for line in geotiff_ds.Landsat_Cloud_Mask.values])
    mask = mask.rename({'dim_0':'stacked_dim'})
    geotiff_dsLS['Landsat_Cloud_Mask'] = mask

    #########################################################################################################
    # Assign descriptive attributes for the cloud mask and the file datetime
    geotiff_ds = geotiff_ds.assign_attrs(
        Cloud_Mask_Bits="Bit 0: Fill\nBit 1: Dilated Cloud\nBit 2: Cirrus (high confidence)\nBit 3: Cloud\nBit 4: Cloud Shadow\n\
    Bit 5: Snow\nBit 6: Clear\n    0: Cloud or Dilated Cloud bits are set\n    1: Cloud and Dilated Cloud bits are not set\nBit 7: Water\n\
    Bits 8-9: Cloud Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High\n\
    Bits 10-11: Cloud Shadow Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High\n\
    Bits 12-13: Snow/Ice Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High\n\
    Bits 14-15: Cirrus Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High")

    geotiff_ds = geotiff_ds.assign_attrs(Datetime=str(time))

    #########################################################################################################
    # Optional filtering by lat/lon
    if coord_bounds:
        geotiff_ds = geotiff_ds.sel(longitude=slice(coord_bounds[0], coord_bounds[1])).sel(latitude=slice(coord_bounds[3], coord_bounds[2]))

    #########################################################################################################
    if save_as=='nc':
        # Convert file into netCDF
        geotiff_ds.to_netcdf(f'{name}.nc')
    else:
        return geotiff_dsLS

In [67]:
"""
Processing of individual .tif files.

Performs a variety of tasks on the data to make it more easy to read and understand.

Attributes:
    tif (str): Path where tif file is located.
    time (datetime or str): Date and time of when the data was collected.
    name (str): Desired name of output file. Just the name, leave out the ".format" part at the end.
    coord_bounds (tuple or list, optional): Coordinate bounds if you wish to filter the data by location. Order should be
                                    (longitude minimum, longitude maximum, latitude minimum, latitude maximum).
    save_as (str, optional): Format to save the processed data in. For now, only 'nc' is built-in. Anything else
                                    simply returns the data.
"""
def process_GOES_tif(tif, time, name, coord_bounds=None, save_as='nc'):
    #########################################################################################################
    # Open file and rename variables
    dsG = rxr.open_rasterio(tif)
    stackedG = dsG.stack(utm_coords=('x', 'y'))
    geotiff_dsG = stackedG.to_dataset('band')

    def stacked_to_latlon(pt):
        return utm_proj(pt[0], pt[1], inverse=True)

    ptsG = list(map(stacked_to_latlon, geotiff_dsG.stacked_dim.values))
    geotiff_dsG.coords['latlon_coords'] = (['latlons', 'coords'], ptsG)
    
    geotiff_dsG = geotiff_dsG.rename({1:'GOES_Blue_Vis_Reflectance',
                                        2:'GOES_Red_Vis_Reflectance', 3:'GOES_NIR_Veggie_Reflectance',
                                        4:'GOES_NIR_SnowIce_Reflectance', 5:'GOES_NIR_CloudParticle_Reflectance',
                                        6:'GOES_LWIR_Brightness_Temp', 7:'GOES_Dirty_LWIR_Brightness_temp'})

    #########################################################################################################
    # Optional filtering by lat/lon
    if coord_bounds:
        geotiff_ds = geotiff_ds.sel(longitude=slice(coord_bounds[0], coord_bounds[1])).sel(latitude=slice(coord_bounds[3], coord_bounds[2]))

    #########################################################################################################
    if save_as=='nc':
        # Convert file into netCDF
        geotiff_ds.to_netcdf(f'{name}.nc')
    else:
        return geotiff_ds

In [61]:
# Runs processing function on all the geotif files
#coord_bounds = [-76.761259, -76.404021, 39.202514, 39.422284]

for i in Landsat_indexes:
    process_Landsat_Sentinel_tif(geotiff_list[i], times_Landsat.value[i], f'../../processed_LS/tif_{i}')

In [ ]:
# Runs processing function on all the geotif files
#coord_bounds = [-76.761259, -76.404021, 39.202514, 39.422284]

for i in Landsat_indexes:
    process_GOES_tif(geotiff_list[i], times_GOES.value[i], f'../../processed_GOES/tif_{i}')

## Sample Code

In [18]:
dsLS = rxr.open_rasterio('../../Landsat_tifs/Landsat_Sentinel_image_0.tif')
dsG = rxr.open_rasterio('../../GOES_tifs/GOES_image_21.tif')
#ds1 = rxr.open_rasterio('../../tifs/GOES_image_0.tif')
#ds2 = rxr.open_rasterio('../../tifs/GOES_image_1.tif')

In [51]:
dsLS

<xarray.DataArray (band: 10, y: 3000, x: 3000)>
array([[[  0.074918,   0.068317, ...,   0.055613,   0.056932],
        [  0.069225,   0.073625, ...,   0.045987,   0.044338],
        ...,
        [  0.148178,   0.153842, ...,   0.147848,   0.144685],
        [  0.145813,   0.145153, ...,   0.15541 ,   0.145785]],

       [[  0.11251 ,   0.10349 , ...,   0.09876 ,   0.090923],
        [  0.127965,   0.139515, ...,   0.0871  ,   0.081545],
        ...,
        [  0.195148,   0.20612 , ...,   0.206037,   0.20172 ],
        [  0.195258,   0.194927, ...,   0.209888,   0.204635]],

       ...,

       [[       nan,        nan, ...,        nan,        nan],
        [       nan,        nan, ...,        nan,        nan],
        ...,
        [-13.271729, -14.002548, ..., -31.120233, -33.450239],
        [-11.799348, -16.036558, ..., -31.439739, -32.938749]],

       [[       nan,        nan, ...,        nan,        nan],
        [       nan,        nan, ...,        nan,        nan],
        ...,
        [ 38.271255,  38.273052, ...,  43.372044,  43.373646],
        [ 38.270977,  38.272774, ...,  43.371796,  43.373398]]])
Coordinates:
  * band         (band) int64 1 2 3 4 5 6 7 8 9 10
  * x            (x) float64 2.92e+05 2.92e+05 2.921e+05 ... 3.82e+05 3.82e+05
  * y            (y) float64 4.372e+06 4.372e+06 ... 4.282e+06 4.282e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      ('SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', '...

In [78]:
stackedLS = dsLS.stack(utm_coords=('x', 'y'))
stackedG = dsG.stack(utm_coords=('x', 'y'))
stackedLS

<xarray.DataArray (band: 10, utm_coords: 9000000)>
array([[  0.0749175 ,   0.069225  ,   0.072745  , ...,   0.1472975 ,
          0.144685  ,   0.145785  ],
       [  0.11251   ,   0.127965  ,   0.1464725 , ...,   0.2026    ,
          0.20172   ,   0.204635  ],
       [  0.2763    ,   0.28697   ,   0.3068525 , ...,   0.2268275 ,
          0.22856   ,   0.2327125 ],
       ...,
       [         nan,          nan,          nan, ..., -21.50701697,
        -22.10128354, -23.15400349],
       [         nan,          nan,          nan, ..., -31.05807109,
        -33.45023946, -32.93874892],
       [         nan,          nan,          nan, ...,  43.37388992,
         43.37364578,  43.37339783]])
Coordinates:
  * band         (band) int64 1 2 3 4 5 6 7 8 9 10
    spatial_ref  int64 0
  * utm_coords   (utm_coords) object MultiIndex
  * x            (utm_coords) float64 2.92e+05 2.92e+05 ... 3.82e+05 3.82e+05
  * y            (utm_coords) float64 4.372e+06 4.372e+06 ... 4.282e+06
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      ('SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', '...

In [102]:
geotiff_dsLS = stackedLS.to_dataset('band')
geotiff_dsG = stackedG.to_dataset('band')
#geotiff_ds1 = ds1.to_dataset('band')
#geotiff_ds2 = ds2.to_dataset('band')

In [103]:
def stacked_to_latlon(pt):
    return utm_proj(pt[0], pt[1], inverse=True)

In [104]:
ptsLS = np.array(list(map(stacked_to_latlon, geotiff_dsLS.utm_coords.values)))
ptsG = np.array(list(map(stacked_to_latlon, geotiff_dsG.utm_coords.values)))

In [106]:
geotiff_dsLS.coords['latlon_coords'] = (['latlons', 'coords'], ptsLS)
geotiff_dsG.coords['latlon_coords'] = (['latlons', 'coords'], ptsG)

In [117]:
geotiff_dsLS['latlon_coords']

<xarray.DataArray 'latlon_coords' (latlons: 9000000, coords: 2)>
array([[-77.4179854 ,  39.47413568],
       [-77.41797605,  39.4738656 ],
       [-77.41796669,  39.47359553],
       ...,
       [-76.35679425,  38.68121463],
       [-76.35678915,  38.68094435],
       [-76.35678404,  38.68067407]])
Coordinates:
    spatial_ref    int64 0
    latlon_coords  (latlons, coords) float64 -77.42 39.47 ... -76.36 38.68
Dimensions without coordinates: latlons, coords

In [108]:
geotiff_dsG

<xarray.Dataset>
Dimensions:        (utm_coords: 9000000, latlons: 9000000, coords: 2)
Coordinates:
    spatial_ref    int64 0
  * utm_coords     (utm_coords) object MultiIndex
  * x              (utm_coords) float64 2.92e+05 2.92e+05 ... 3.82e+05 3.82e+05
  * y              (utm_coords) float64 4.372e+06 4.372e+06 ... 4.282e+06
    latlon_coords  (latlons, coords) float64 -77.42 39.47 ... -76.36 38.68
Dimensions without coordinates: latlons, coords
Data variables:
    1              (utm_coords) float64 0.114 0.114 0.114 ... 0.1353 0.1353
    2              (utm_coords) float64 0.07888 0.07888 ... 0.09548 0.09548
    3              (utm_coords) float64 0.1192 0.1192 0.1192 ... 0.0989 0.0989
    4              (utm_coords) float64 0.08303 0.08303 ... 0.04957 0.04957
    5              (utm_coords) float64 0.06227 0.06227 ... 0.05128 0.05128
    6              (utm_coords) float64 282.9 282.9 282.9 ... 272.1 272.1 272.1
    7              (utm_coords) float64 281.6 281.6 281.6 ... 270.8 270.8 270.8
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      ('CMI_C01', 'CMI_C02', 'CMI_C03', 'CMI_C05', 'CMI_C06', '...

In [118]:
# For GOES tifs
geotiff_dsG = geotiff_dsG.rename({1:'GOES_Blue_Vis_Reflectance',
                               2:'GOES_Red_Vis_Reflectance', 3:'GOES_NIR_Veggie_Reflectance',
                               4:'GOES_NIR_SnowIce_Reflectance', 5:'GOES_NIR_CloudParticle_Reflectance',
                               6:'GOES_LWIR_Brightness_Temp', 7:'GOES_Dirty_LWIR_Brightness_temp'})

In [119]:
# For Landsat/Sentinel tifs
geotiff_dsLS = geotiff_dsLS.rename({1:'Landsat_Blue_Vis_Sfc_Reflectance', 2:'Landsat_Red_Vis_Sfc_Reflectance',
                                3:'Landsat_NIR_Sfc_Reflectance', 4:'Landsat_SWIR1_Sfc_Reflectance',
                               5:'Landsat_SWIR2_Sfc_Reflectance', 6:'Landsat_LST',
                               7:'Landsat_Cloud_Mask', 8:'VV_SAR', 9:'VH_SAR', 10:'SAR_angle'})

In [120]:
geotiff_dsG

<xarray.Dataset>
Dimensions:                             (utm_coords: 9000000, latlons: 9000000,
                                         coords: 2)
Coordinates:
    spatial_ref                         int64 0
  * utm_coords                          (utm_coords) object MultiIndex
  * x                                   (utm_coords) float64 2.92e+05 ... 3.8...
  * y                                   (utm_coords) float64 4.372e+06 ... 4....
    latlon_coords                       (latlons, coords) float64 -77.42 ... ...
Dimensions without coordinates: latlons, coords
Data variables:
    GOES_Blue_Vis_Reflectance           (utm_coords) float64 0.114 ... 0.1353
    GOES_Red_Vis_Reflectance            (utm_coords) float64 0.07888 ... 0.09548
    GOES_NIR_Veggie_Reflectance         (utm_coords) float64 0.1192 ... 0.0989
    GOES_NIR_SnowIce_Reflectance        (utm_coords) float64 0.08303 ... 0.04957
    GOES_NIR_CloudParticle_Reflectance  (utm_coords) float64 0.06227 ... 0.05128
    GOES_LWIR_Brightness_Temp           (utm_coords) float64 282.9 ... 272.1
    GOES_Dirty_LWIR_Brightness_temp     (utm_coords) float64 281.6 ... 270.8
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      ('CMI_C01', 'CMI_C02', 'CMI_C03', 'CMI_C05', 'CMI_C06', '...

In [121]:
# Baltimore-ish
coord_bounds = [-76.761259, -76.404021, 39.202514, 39.422284]

In [122]:
#gds = geotiff_ds.sel(longitude=slice(coord_bounds[0], coord_bounds[1])).sel(latitude=slice(coord_bounds[3], coord_bounds[2]))

In [123]:
mask = xr.DataArray([to_binary_string(x) for x in geotiff_dsLS.Landsat_Cloud_Mask.values])

In [124]:
mask

<xarray.DataArray (dim_0: 9000000)>
array(['1101011100001100', '1101011100001100', '1101011100001100', ...,
       '1101011100001100', '1101011100001100', '1101011100001100'],
      dtype='<U16')
Dimensions without coordinates: dim_0

In [125]:
mask = mask.rename({'dim_0':'utm_coords'})

In [126]:
geotiff_dsLS['Landsat_Cloud_Mask'] = mask

In [127]:
geotiff_dsLS

<xarray.Dataset>
Dimensions:                           (utm_coords: 9000000, latlons: 9000000,
                                       coords: 2)
Coordinates:
    spatial_ref                       int64 0
  * utm_coords                        (utm_coords) object MultiIndex
  * x                                 (utm_coords) float64 2.92e+05 ... 3.82e+05
  * y                                 (utm_coords) float64 4.372e+06 ... 4.28...
    latlon_coords                     (latlons, coords) float64 -77.42 ... 38.68
Dimensions without coordinates: latlons, coords
Data variables:
    Landsat_Blue_Vis_Sfc_Reflectance  (utm_coords) float64 0.07492 ... 0.1458
    Landsat_Red_Vis_Sfc_Reflectance   (utm_coords) float64 0.1125 ... 0.2046
    Landsat_NIR_Sfc_Reflectance       (utm_coords) float64 0.2763 ... 0.2327
    Landsat_SWIR1_Sfc_Reflectance     (utm_coords) float64 0.2057 ... 0.1351
    Landsat_SWIR2_Sfc_Reflectance     (utm_coords) float64 0.1595 ... 0.1401
    Landsat_LST                       (utm_coords) float64 262.5 262.5 ... 246.7
    Landsat_Cloud_Mask                (utm_coords) <U16 '1101011100001100' .....
    VV_SAR                            (utm_coords) float64 nan nan ... -23.15
    VH_SAR                            (utm_coords) float64 nan nan ... -32.94
    SAR_angle                         (utm_coords) float64 nan nan ... 43.37
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      ('SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', '...

In [128]:
geotiff_dsLS = geotiff_dsLS.assign_attrs(
    Cloud_Mask_Bits="Bit 0: Fill\nBit 1: Dilated Cloud\nBit 2: Cirrus (high confidence)\nBit 3: Cloud\nBit 4: Cloud Shadow\n\
Bit 5: Snow\nBit 6: Clear\n    0: Cloud or Dilated Cloud bits are set\n    1: Cloud and Dilated Cloud bits are not set\nBit 7: Water\n\
Bits 8-9: Cloud Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High\n\
Bits 10-11: Cloud Shadow Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High\n\
Bits 12-13: Snow/Ice Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High\n\
Bits 14-15: Cirrus Confidence\n    0: None\n    1: Low\n    2: Medium\n    3: High")

In [129]:
geotiff_dsLS = geotiff_dsLS.assign_attrs(Datetime=str(times_Landsat.value[0]))

In [130]:
geotiff_dsLS

<xarray.Dataset>
Dimensions:                           (utm_coords: 9000000, latlons: 9000000,
                                       coords: 2)
Coordinates:
    spatial_ref                       int64 0
  * utm_coords                        (utm_coords) object MultiIndex
  * x                                 (utm_coords) float64 2.92e+05 ... 3.82e+05
  * y                                 (utm_coords) float64 4.372e+06 ... 4.28...
    latlon_coords                     (latlons, coords) float64 -77.42 ... 38.68
Dimensions without coordinates: latlons, coords
Data variables:
    Landsat_Blue_Vis_Sfc_Reflectance  (utm_coords) float64 0.07492 ... 0.1458
    Landsat_Red_Vis_Sfc_Reflectance   (utm_coords) float64 0.1125 ... 0.2046
    Landsat_NIR_Sfc_Reflectance       (utm_coords) float64 0.2763 ... 0.2327
    Landsat_SWIR1_Sfc_Reflectance     (utm_coords) float64 0.2057 ... 0.1351
    Landsat_SWIR2_Sfc_Reflectance     (utm_coords) float64 0.1595 ... 0.1401
    Landsat_LST                       (utm_coords) float64 262.5 262.5 ... 246.7
    Landsat_Cloud_Mask                (utm_coords) <U16 '1101011100001100' .....
    VV_SAR                            (utm_coords) float64 nan nan ... -23.15
    VH_SAR                            (utm_coords) float64 nan nan ... -32.94
    SAR_angle                         (utm_coords) float64 nan nan ... 43.37
Attributes:
    AREA_OR_POINT:    Area
    scale_factor:     1.0
    add_offset:       0.0
    long_name:        ('SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10',...
    Cloud_Mask_Bits:  Bit 0: Fill\nBit 1: Dilated Cloud\nBit 2: Cirrus (high ...
    Datetime:         2023-01-11 10:46:48.979000

In [ ]:
# Export the data
#geotiff_ds.rio.to_raster("first.tif")
geotiff_ds.to_netcdf('first.nc')

In [27]:
xr.load_dataset('first.nc')

<xarray.Dataset>
Dimensions:                             (latitude: 6400, longitude: 6400)
Coordinates:
  * longitude                           (longitude) float64 -77.51 ... -75.79
  * latitude                            (latitude) float64 39.9 39.9 ... 38.17
Data variables: (12/15)
    Landsat_Blue_Vis_Sfc_Reflectance    (latitude, longitude) float64 nan ......
    Landsat_Red_Vis_Sfc_Reflectance     (latitude, longitude) float64 nan ......
    Landsat_NIR_Sfc_Reflectance         (latitude, longitude) float64 nan ......
    Landsat_SWIR1_Sfc_Reflectance       (latitude, longitude) float64 nan ......
    Landsat_SWIR2_Sfc_Reflectance       (latitude, longitude) float64 nan ......
    Landsat_LST                         (latitude, longitude) float64 nan ......
    ...                                  ...
    GOES_NIR_Veggie_Reflectance         (latitude, longitude) float64 0.1299 ...
    GOES_NIR_SnowIce_Reflectance        (latitude, longitude) float64 0.06935...
    GOES_NIR_CloudParticle_Reflectance  (latitude, longitude) float64 0.06691...
    GOES_LWIR_Brightness_Temp           (latitude, longitude) float64 274.2 ....
    GOES_Dirty_LWIR_Brightness_temp     (latitude, longitude) float64 273.1 ....
    spatial_ref                         int64 0
Attributes:
    AREA_OR_POINT:    Area
    scale_factor:     1.0
    add_offset:       0.0
    long_name:        ['SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10',...
    Cloud_Mask_Bits:  Bit 0: Fill\nBit 1: Dilated Cloud\nBit 2: Cirrus (high ...
    Datetime:         2023-01-11 10:46:33.089500

In [ ]:
######################################################
# End of tif processing part
# Below is gridding of Baltimore

In [31]:
grid_lats = [39.2576, 39.3147, 39.3718]
grid_lons = [-76.5299, -76.6204, -76.7108]

In [37]:
grid1 = geotiff_ds.sel(longitude=slice(grid_lons[2], grid_lons[1])).sel(latitude=slice(grid_lats[2], grid_lats[1]))
grid2 = geotiff_ds.sel(longitude=slice(grid_lons[1], grid_lons[0])).sel(latitude=slice(grid_lats[2], grid_lats[1]))
grid3 = geotiff_ds.sel(longitude=slice(grid_lons[2], grid_lons[1])).sel(latitude=slice(grid_lats[1], grid_lats[0]))
grid4 = geotiff_ds.sel(longitude=slice(grid_lons[1], grid_lons[0])).sel(latitude=slice(grid_lats[1], grid_lats[0]))

In [38]:
grid4

<xarray.Dataset>
Dimensions:                             (latitude: 212, longitude: 336)
Coordinates:
  * longitude                           (longitude) float64 -76.62 ... -76.53
  * latitude                            (latitude) float64 39.31 39.31 ... 39.26
    spatial_ref                         int64 0
Data variables: (12/14)
    Landsat_Blue_Vis_Sfc_Reflectance    (latitude, longitude) float64 ...
    Landsat_Red_Vis_Sfc_Reflectance     (latitude, longitude) float64 ...
    Landsat_NIR_Sfc_Reflectance         (latitude, longitude) float64 ...
    Landsat_SWIR1_Sfc_Reflectance       (latitude, longitude) float64 ...
    Landsat_SWIR2_Sfc_Reflectance       (latitude, longitude) float64 ...
    Landsat_LST                         (latitude, longitude) float64 ...
    ...                                  ...
    GOES_Red_Vis_Reflectance            (latitude, longitude) float64 ...
    GOES_NIR_Veggie_Reflectance         (latitude, longitude) float64 ...
    GOES_NIR_SnowIce_Reflectance        (latitude, longitude) float64 ...
    GOES_NIR_CloudParticle_Reflectance  (latitude, longitude) float64 ...
    GOES_LWIR_Brightness_Temp           (latitude, longitude) float64 ...
    GOES_Dirty_LWIR_Brightness_temp     (latitude, longitude) float64 ...
Attributes:
    AREA_OR_POINT:    Area
    scale_factor:     1.0
    add_offset:       0.0
    long_name:        ('SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10',...
    Cloud_Mask_Bits:  Bit 0: Fill\nBit 1: Dilated Cloud\nBit 2: Cirrus (high ...
    Datetime:         2023-01-11 10:46:33.089500